# Day 17
https://adventofcode.com/2018/day/17

In [1]:
import aocd
data = aocd.get_data(year=2018, day=17)

In [2]:
from collections import deque
from dataclasses import dataclass
import regex as re

In [3]:
re_vertical = re.compile(r'x=(\d+), y=(\d+)..(\d+)')
re_horizontal = re.compile(r'y=(\d+), x=(\d+)..(\d+)')

In [4]:
def read_sand(text):
    sand = set()

    for x, min_y, max_y in [tuple(map(int, val)) for val in re_vertical.findall(text)]:
        for y in range(min_y, max_y+1):
            sand.add(complex(x, y))

    for y, min_x, max_x in [tuple(map(int, val)) for val in re_horizontal.findall(text)]:
        for x in range(min_x, max_x+1):
            sand.add(complex(x, y))

    return sand

In [5]:
UP = -1j
DOWN = 1j
LEFT = -1
RIGHT = 1

In [6]:
def fill_horizontally(start, occupied, water):
    y = start.imag
    occ_in_row = {int(o.real) for o in occupied if o.imag == y}
    
    if not occ_in_row:
        return False
    
    min_x = min(x for x in occ_in_row)
    max_x = max(x for x in occ_in_row)
    
    x = int(start.real)
    if min_x > x or max_x < x:
        return False
    
    while x > min_x:
        if complex(x, y+1) not in occupied:
            return False
        elif complex(x-1, y) in occupied:
            min_x = x
        x -= 1
    
    x = int(start.real)
    while x < max_x:
        if complex(x, y+1) not in occupied:
            return False
        elif complex(x+1, y) in occupied:
            max_x = x
        x += 1
    
    for x in range(min_x, max_x+1):
        water.add(complex(x, y))
    return True

In [7]:
def flow(sand):
    min_y = min(s.imag for s in sand)
    max_y = max(s.imag for s in sand)
    
    flows = deque()
    flows.append((DOWN, 500))
    visited = set()
    water = set()
    
    while flows:
        direct, pos = flows.popleft()
        
        if (direct, pos) not in flows:
            visited.add(pos)
            occupied = sand.union(water)
            
            if direct == DOWN:
                if (pos + DOWN) in occupied:
                    if fill_horizontally(pos, occupied, water):
                        flows.append((DOWN, pos+UP))
                    else:
                        flows.append((LEFT, pos))
                        flows.append((RIGHT, pos))
                elif (pos + DOWN).imag <= max_y:
                    flows.append((DOWN, pos+DOWN))
            
            elif (pos + DOWN) not in occupied:
                flows.append((DOWN, pos))
            
            elif (pos + direct) not in occupied:
                flows.append((direct, pos + direct))
        
    visited = {i for i in visited if i.imag >= min_y}
    return water, visited

In [8]:
sand = read_sand(data)
stationary, flowing = flow(sand)
p1 = len(stationary.union(flowing))
p2 = len(stationary)
print('Part 1: {}'.format(p1))
print('Part 2: {}'.format(p2))

Part 1: 39557
Part 2: 32984
